In [ ]:
%cd ..
%pwd

In [ ]:
import json
import logging
from pathlib import Path

from robust_dst.evaluation import get_metrics
from robust_dst.scoring_utils import flatten_metrics_dict, setup_sgd_evaluation, setup_sgd_evaluator_inputs
from robust_dst.utils import (
    infer_data_version_from_path,
    infer_schema_variant_from_path,
)


logger = logging.getLogger(__name__)

In [ ]:
first_file_path = "data/raw/original/test/dialogues_001.json"
hyp_path = "experiments/metrics_and_dialogues_sdt_best.json"

with open(first_file_path, "r") as file:
    refs = json.load(file)



In [ ]:
# with open(hyp_path, "r") as file:
#     hyps = json.load(file)


# Read everything in "experiments/sdt" folder
# Take all files which start with dialogues_ and end with .json
# Read them and convert them to a list of dictionaries, joining them all together to a single dictionary
# called hyps
hyps = []
hyp_dir = Path("experiments/pytod")
for file_path in hyp_dir.glob("dialogues_*.json"):
    with open(file_path, "r") as file:
        data = json.load(file)
        hyps.extend(data)

hyps = {d["dialogue_id"]: d for d in hyps}

In [ ]:
sgd_evaluator_inputs = setup_sgd_evaluator_inputs(Path('data/raw/original/test'))

In [ ]:
all_metrics_aggregate, per_frame_metric = get_metrics(
    dataset_ref=sgd_evaluator_inputs["dataset_ref"],
    dataset_hyp=hyps,
    service_schemas=sgd_evaluator_inputs["eval_services"],
    in_domain_services=sgd_evaluator_inputs["in_domain_services"],
    fuzzy_threshold=0.8
)

In [ ]:
services = ["#ALL_SERVICES", "#SEEN_SERVICES", "#UNSEEN_SERVICES"]
metrics = ["joint_goal_accuracy", "consistency_adjusted_joint_goal_accuracy", "correct_turns"]
for service in services:
    for metric in metrics:
        print(f"{service}/{metric}: {all_metrics_aggregate[service][metric]}")